In [2]:
import pandas as pd
import numpy as np
from os.path import join as PJOIN

In [6]:
PREFIX = "comments"
PROJECT_NAME = "libpng"
SUFFIX = "all_marked"
FILE_NAME = PREFIX + "_" + PROJECT_NAME + "_" + SUFFIX + ".xlsx"
FILE_PATH = PJOIN("DATA","ANNOTATED",FILE_NAME)
OUTPUT_FILE_PATH = PJOIN("DATA","GENERATED",FILE_NAME)
MAP = {'U':'U', 'PU':'P', 'NU':'N'}
THRESHOLD = 10

# Rules

In [7]:
# c is a vector of size 31 [comment text, C1, C2, ......., C30]
def get_label(c):
    #IF C18 OR C19 OR C20  OR  C21  OR C22 OR C28 OR C29 THEN  U
    if c[18] or c[19] or c[20] or c[21] or c[22] or c[28] or c[29]:
        return 'U'
    #IF C9 AND C3 THEN U
    if c[9] and c[3]:
        return 'U'
    #IF C11 AND C3 THEN U
    if c[11] and c[3]:
        return 'U'
    #IF (C25 OR C23 OR C26 OR C27 ) AND C3 THEN U
    if (c[25] or c[23] or c[26] or c[27]) and c[3]:
        return 'U'
    #IF (C25 OR C23 OR C26 OR C27) AND C4 AND (C9  OR C11) THEN U
    if (c[25] or c[23] or c[26] or c[27]) and c[4] and (c[9] or c[11]):
        return 'U'
    #IF C10 AND C15  AND C3 THEN U
    if c[10] and c[15] and c[3]:
        return 'U'
    #IF C8 AND C15  AND C3 THEN U
    if c[8] and c[15] and c[3]:
        return 'U'
    #IF (C18 OR C19 OR C20 OR C21) AND C17 THEN U
    if (c[18] or c[19] or c[20] or c[21]) and c[17]:
        return 'U'
    #Low Problem Domain AND Low Scope AND Concepts don't match Symbols = U
    if c[10] and c[15] and c[3]:
        return 'U'
    #Low Problem Domain AND Low Scope AND Concepts Partially Match = U
    if c[10] and c[15] and c[4]:
        return 'U'
    
    #IF C9 AND (C4 OR C5) THEN PU
    if c[9] and (c[4] or c[5]):
        return 'PU'
    #IF C11 AND (C4  OR C5) THEN PU
    if c[11] and (c[4] or c[5]):
        return 'PU'
    #IF C10 AND C14  AND C3 THEN PU
    if c[1] and c[14] and c[3]:
        return 'PU'
    #IF C8 AND C14  AND C3 THEN PU
    if c[8] and c[14] and c[3]:
        return 'PU'
    #IF C17 AND NOT (C18 OR C19 OR C20 OR C21)  THEN PU
    if c[17] and not((c[18] or c[19] or c[20] or c[21])):
        return 'PU'
    #Low  Problem Domain AND Concepts Partially Match = PU
    if c[10] and c[4]:
        return 'PU'
    #Low Program Domain AND Concepts Partially Match = PU
    if c[8] and c[4]:
        return 'PU'
    
    #IF C12 OR C16 THEN  NU
    if c[12] or c[16]:
        return 'NU'
    #IF (C8 OR C9)  AND C1 THEN NU
    if (c[8] or c[9]) and c[1]:
        return 'NU'
    #IF (C10 OR C11) AND C1 THEN NU
    if (c[10] or c[11]) and c[1]:
        return 'NU'
    #IF C10 AND C15  AND C1 THEN U
    if c[10] and c[15] and c[1]:
        return 'NU'
    #IF C8 AND C15  AND C1 THEN U
    if c[8] and c[15] and c[1]:
        return 'NU'
    
    #print("###NOMATCH")
    return 'NU'
    


In [8]:
if FILE_PATH[-3:] == 'csv':
    exl_file = pd.read_csv(FILE_PATH,delimiter='\t')
else:
    exl_file = pd.read_excel(FILE_PATH)
exl_file.head()

,Filename,Comment text,Start line,End line,No. of words,Program Domain Concepts,Problem Domain Concepts,Copyright/License,Bug/Fix/Patch/Version,Build,...,C22,C23,C24,C25,C26,C27,C28,C29,C30,Score
0,repos/libpng-code/pngset.c,pngset.c - storage of image information into i...,2,17,116,"{'storage': ['storag', 'Time Complexity / Spac...","['image', 'libpng']",True,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,U
1,repos/libpng-code/pngset.c,override with app values,62,61,4,"{'values': ['valu', 'Data-Structure and its Co...",[],False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P
2,repos/libpng-code/pngset.c,FLOATING_POINT,133,133,1,{},[],False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
3,repos/libpng-code/pngset.c,cHRM,135,135,1,{},[],False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P
4,repos/libpng-code/pngset.c,eXIf,182,182,1,{},[],False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P


In [9]:
exl_np = np.array(exl_file)

In [10]:
classes = exl_np[:,[1]+list(range(16,46))]
classes[0]

array(['pngset.c - storage of image information into info struct\n *\n * Copyright (c) 2018 Cosmin Truta\n * Copyright (c) 1998-2018 Glenn Randers-Pehrson\n * Copyright (c) 1996-1997 Andreas Dilger\n * Copyright (c) 1995-1996 Guy Eric Schalnat, Group 42, Inc.\n *\n * This code is released under the libpng license.\n * For conditions of distribution and use, see the disclaimer\n * and license in png.h\n *\n * The functions here are used during reads to store data from the file\n * into the info struct, and during writes to store application data\n * into the info struct for writing into the file.  This abstracts the\n * info struct and allows us to change the structure in the future.',
       nan, nan, nan, 1.0, nan, nan, nan, nan, 1.0, nan, 1.0, nan, nan,
       1.0, nan, 1.0, nan, nan, nan, nan, 1.0, nan, nan, nan, nan, nan,
       nan, 1.0, nan, nan], dtype=object)

In [11]:
for j, c in enumerate(classes):
    for i in range(1,31):
        try:
            if c[i] != c[i]:
                c[i] = False
#             elif c[i] == 'False':
#                 c[i] = False
#             elif c[i] == 'True':
#                 c[i] = True
            elif int(c[i]) == 1:
                c[i] = True
            else:
                c[i] = False
        except:
            print(j, i)

In [12]:
labels = []
for c in classes:
    labels.append(MAP[get_label(c)])

In [13]:
exl_file['Calculated Score'] = labels

In [14]:
if FILE_PATH[-3:] == 'csv':
    exl_file.to_csv(OUTPUT_FILE_PATH,index=False)
else:
    exl_file.to_excel(OUTPUT_FILE_PATH, index=False)

# Analysis

In [15]:
intuitive_labels = exl_np[:,-1]

In [16]:
lables_np = np.array(labels)

In [172]:
num_nan = np.sum(np.array([x !=x for x in intuitive_labels]))

In [173]:
sum(lables_np == intuitive_labels)/(len(lables_np) - num_nan), (len(lables_np) - num_nan)

(0.08846584546472565, 893)

In [174]:
losses = {}

In [175]:
for i in range(len(intuitive_labels)):
    if intuitive_labels[i]!=intuitive_labels[i]:
        continue
    li = intuitive_labels[i]
    lc = lables_np[i]
    if li not in losses:
        losses[li] = {}
    if lc not in losses[li]:
        losses[li][lc] = 0
    losses[li][lc] += 1

In [176]:
losses

{'Pu': {'N': 125, 'P': 6, 'U': 21},
 'U': {'N': 112, 'P': 41, 'U': 79},
 'Us': {'N': 507, 'P': 2}}

In [177]:
sm = 0
for i in range(len(intuitive_labels)):
    if intuitive_labels[i] != intuitive_labels[i]:
        continue
    if intuitive_labels[i][0] == lables_np[i][0]:
        sm+=1

In [178]:
sm/(len(lables_np) - num_nan)

0.09518477043673013

In [17]:
losses = {}
def get_hash(classes):
    hsh = []
    for i, c in enumerate(classes):
        if c == True:
            hsh.append(i+1)
    return str(hsh)

for i in range(len(intuitive_labels)):
    if intuitive_labels[i]!=intuitive_labels[i]:
        continue
    li = intuitive_labels[i][0]
    lc = lables_np[i]
    if li not in losses:
        losses[li] = {}
    if lc not in losses[li]:
        losses[li][lc] = {}
    if li==lc:
        continue
    myhash = get_hash(classes[i][1:])
    if myhash not in losses[li][lc]:
        losses[li][lc][myhash] = 0
    losses[li][lc][myhash] += 1

In [18]:
alls = []
for li, v1 in losses.items():
    for lc, v2 in v1.items():
        curr = 0
        tops_curr = None
        for  myhash, ct in v2.items():
            alls.append([li,lc,myhash,ct])
#             if ct > curr:
#                 curr = ct
#                 tops_curr = [li, lc, myhash, ct]
#         if tops_curr is not None:
#             tops.append(tops_curr)

In [19]:
print("Inutinitive Label | Calculated Label | Classes which have 1 | Number of such mismatch")
alls = sorted(alls,key=lambda x:(x[3],x[0],x[1]),reverse=True)
alls

Inutinitive Label | Calculated Label | Classes which have 1 | Number of such mismatch


[['U', 'N', '[3, 7, 8]', 114],
 ['U', 'N', '[3, 6, 7, 13]', 66],
 ['U', 'N', '[3, 8, 10]', 61],
 ['P', 'N', '[3, 7, 8]', 59],
 ['N', 'U', '[3, 7, 8, 13, 15]', 57],
 ['P', 'N', '[3, 7, 8, 13]', 51],
 ['P', 'U', '[3, 7, 9]', 40],
 ['P', 'N', '[3, 6, 7, 13]', 40],
 ['N', 'U', '[2, 3, 7, 8, 13, 15]', 39],
 ['U', 'N', '[3, 7, 8, 13]', 33],
 ['P', 'N', '[3, 6, 7, 13, 15]', 25],
 ['P', 'N', '[1, 6, 7, 13, 15]', 25],
 ['P', 'U', '[3, 7, 8, 13, 15]', 19],
 ['U', 'N', '[3, 6, 7, 13, 15]', 17],
 ['U', 'N', '[3, 6, 10]', 17],
 ['U', 'P', '[4, 7, 9]', 16],
 ['U', 'N', '[3, 6, 7]', 16],
 ['U', 'P', '[4, 9, 11]', 13],
 ['U', 'N', '[3, 6, 10, 13]', 13],
 ['P', 'U', '[3, 9, 10]', 12],
 ['U', 'P', '[4, 9, 10]', 11],
 ['U', 'N', '[1, 7, 8, 15]', 10],
 ['P', 'N', '[3, 8, 10]', 10],
 ['U', 'P', '[4, 9, 11, 14]', 9],
 ['P', 'U', '[3, 6, 10, 13, 15]', 9],
 ['P', 'N', '[3, 6, 10, 13]', 8],
 ['U', 'N', '[3, 8, 10, 13]', 7],
 ['P', 'U', '[3, 7, 8, 15]', 7],
 ['P', 'N', '[1, 6, 10, 13, 15]', 7],
 ['N', 'U', '[3,

In [20]:
import csv
with open('DATA/ANNOTATED/rule_diffs/'+PROJECT_NAME+".csv",'w') as f:
    writer = csv.writer(f)
    writer.writerow(["Intuitive Label","Calculated Label", "Classes which have 1 marked","Number of Mismatches", "Name"])
    for i, row in enumerate(alls):
        if row[3] < THRESHOLD:
            break
        writer.writerow(row + ["Case "+str(i+1)])

In [21]:
alls

[['U', 'N', '[3, 7, 8]', 114],
 ['U', 'N', '[3, 6, 7, 13]', 66],
 ['U', 'N', '[3, 8, 10]', 61],
 ['P', 'N', '[3, 7, 8]', 59],
 ['N', 'U', '[3, 7, 8, 13, 15]', 57],
 ['P', 'N', '[3, 7, 8, 13]', 51],
 ['P', 'U', '[3, 7, 9]', 40],
 ['P', 'N', '[3, 6, 7, 13]', 40],
 ['N', 'U', '[2, 3, 7, 8, 13, 15]', 39],
 ['U', 'N', '[3, 7, 8, 13]', 33],
 ['P', 'N', '[3, 6, 7, 13, 15]', 25],
 ['P', 'N', '[1, 6, 7, 13, 15]', 25],
 ['P', 'U', '[3, 7, 8, 13, 15]', 19],
 ['U', 'N', '[3, 6, 7, 13, 15]', 17],
 ['U', 'N', '[3, 6, 10]', 17],
 ['U', 'P', '[4, 7, 9]', 16],
 ['U', 'N', '[3, 6, 7]', 16],
 ['U', 'P', '[4, 9, 11]', 13],
 ['U', 'N', '[3, 6, 10, 13]', 13],
 ['P', 'U', '[3, 9, 10]', 12],
 ['U', 'P', '[4, 9, 10]', 11],
 ['U', 'N', '[1, 7, 8, 15]', 10],
 ['P', 'N', '[3, 8, 10]', 10],
 ['U', 'P', '[4, 9, 11, 14]', 9],
 ['P', 'U', '[3, 6, 10, 13, 15]', 9],
 ['P', 'N', '[3, 6, 10, 13]', 8],
 ['U', 'N', '[3, 8, 10, 13]', 7],
 ['P', 'U', '[3, 7, 8, 15]', 7],
 ['P', 'N', '[1, 6, 10, 13, 15]', 7],
 ['N', 'U', '[3,

# Extend Annotations with Diff

In [22]:
alls_map = {}
for i, row in enumerate(alls):
    if row[3] < THRESHOLD:
        break
    alls_map[(row[0],row[1],row[2])] = i+1
alls_map

{('N', 'U', '[2, 3, 7, 8, 13, 15]'): 9,
 ('N', 'U', '[3, 7, 8, 13, 15]'): 5,
 ('P', 'N', '[1, 6, 7, 13, 15]'): 12,
 ('P', 'N', '[3, 6, 7, 13, 15]'): 11,
 ('P', 'N', '[3, 6, 7, 13]'): 8,
 ('P', 'N', '[3, 7, 8, 13]'): 6,
 ('P', 'N', '[3, 7, 8]'): 4,
 ('P', 'N', '[3, 8, 10]'): 23,
 ('P', 'U', '[3, 7, 8, 13, 15]'): 13,
 ('P', 'U', '[3, 7, 9]'): 7,
 ('P', 'U', '[3, 9, 10]'): 20,
 ('U', 'N', '[1, 7, 8, 15]'): 22,
 ('U', 'N', '[3, 6, 10, 13]'): 19,
 ('U', 'N', '[3, 6, 10]'): 15,
 ('U', 'N', '[3, 6, 7, 13, 15]'): 14,
 ('U', 'N', '[3, 6, 7, 13]'): 2,
 ('U', 'N', '[3, 6, 7]'): 17,
 ('U', 'N', '[3, 7, 8, 13]'): 10,
 ('U', 'N', '[3, 7, 8]'): 1,
 ('U', 'N', '[3, 8, 10]'): 3,
 ('U', 'P', '[4, 7, 9]'): 16,
 ('U', 'P', '[4, 9, 10]'): 21,
 ('U', 'P', '[4, 9, 11]'): 18}

In [23]:
diff_names = []
diffs_map = {}
for i in range(len(intuitive_labels)):
    if intuitive_labels[i]!=intuitive_labels[i]:
        diff_names.append(None)
        continue
    li = intuitive_labels[i][0]
    lc = lables_np[i]
    myhash = get_hash(classes[i][1:])
    if (li, lc, myhash) not in alls_map:
        diff_names.append(None)
        continue
    diff_names.append("Case " + str(alls_map[(li, lc, myhash)]))
    if alls_map[(li, lc, myhash)] not in diffs_map:
        diffs_map[alls_map[(li, lc, myhash)]] = []
    diffs_map[alls_map[(li, lc, myhash)]].append(i)

In [24]:
exl_file['Mismatch Case'] = diff_names

In [25]:
DIFFS_FILE = FILE_NAME
if DIFFS_FILE[-3:] == 'csv':
    DIFFS_FILE = DIFFS_FILE[:-3] + 'xlsx'

In [26]:
writer = pd.ExcelWriter("DATA/ANNOTATED/rule_diffs/"+DIFFS_FILE, engine = 'xlsxwriter')

In [27]:
exl_file.to_excel(writer,index=False)

In [28]:
for k,v in diffs_map.items():
    df = pd.DataFrame(columns=exl_file.columns)
    for i, j in enumerate(v):
        df.loc[i] = exl_file.loc[j]
    df.to_excel(writer,index=False,sheet_name='Case '+str(k))

In [29]:
writer.save()
writer.close()